In [1]:
import sys, os, time
import pyzx as zx
from benchmarking import Benchmark
import numpy as np
import argparse
import time
from distutils.util import strtobool
from typing import Optional

import gymnasium as gym
import gym_zx

import numpy as np
import torch
from torch_geometric.data import Batch, Data
from rl_agent import AgentGNN

global device




In [2]:
b = Benchmark()

In [3]:
b.load_circuits(
    dirname=os.path.join("results", "circuits", "Original"),
    group_name="fast",
)
b.load_circuits(
    dirname=os.path.join("results", "circuits", "nrscm"),
    group_name="fast",
    simp_strategy="NRSCM",
)
b.load_circuits(
    dirname=os.path.join("results", "circuits", "rl-zx-opt"),
    group_name="fast",
    simp_strategy="RL-ZX",
)
b.load_circuits(
    dirname=os.path.join("results", "circuits", "gflow-cflow-opt"),
    group_name="fast",
    simp_strategy="gflow+cflow",
)



In [4]:
f = b.df(groups=["fast"], routines=["all"], funcs=["all"], atts=["Qubits","Gates","Single Qubit", "2Q Count"], json_file="results/best_random_stats.json")

In [5]:

def basic_optimise(c):
    c1 = zx.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2 # As this optimisation algorithm is targetted at reducting H-gates, we use the circuit with the smaller 2-qubit gate count here, either using SWAP rules or not.
    return c1

def flow_opt(c):
    c.to_basic_gates()
    g = c.to_graph()
    zx.teleport_reduce(g)
    zx.to_graph_like(g)
    zx.flow_2Q_simp(g)
    c2 = zx.extract_simple(g, up_to_perm=True).to_basic_gates()
    return basic_optimise(c2)


In [6]:
#b.add_simplification_func(func=rl_zx, name="RL-ZX", groups_to_run=None, verify=False)
b.add_simplification_func(func=flow_opt, name="cflow-opt", groups_to_run=["fast"], verify=False, rerun=False)


b.run(funcs_to_run=["cflow-opt"], groups_to_run=["fast"], verify=True, rerun=True)

b.show_attributes()

Processing cflow-opt on vbe_adder_3                                   : 100%|██████████| 33/33 [01:04<00:00,  1.97s/it]
Processing cflow-opt on adder_8                                       :  15%|█▌        | 5/33 [00:33<02:37,  5.63s/it]

Circuit resulting from cflow-opt on adder_8 not verified


Processing cflow-opt on csla_mux_3_original                           :  30%|███       | 10/33 [00:42<00:42,  1.87s/it]

Circuit resulting from cflow-opt on csla_mux_3_original not verified


Processing cflow-opt on ham15-low                                     :  55%|█████▍    | 18/33 [00:59<00:48,  3.22s/it]

...

Processing cflow-opt on vbe_adder_3                                   :  97%|█████████▋| 32/33 [01:15<00:00,  1.87it/s]

Circuit resulting from cflow-opt on vbe_adder_3 not verified


Processing cflow-opt on vbe_adder_3                                   : 100%|██████████| 33/33 [01:15<00:00,  2.29s/it]

Circuit attributes:  ['Qubits', 'Gates', 'Single Qubit', '2Q Count', 'T Count', 't_opt', 'depth']
Loaded functions:  ['cflow-opt']
Loaded routines:  ['NRSCM', 'gflow+cflow', 'RL-ZX']
Loaded circuit groups:  ['fast']


,Original,NRSCM,RL-ZX,cflow-opt,gflow+cflow
fast,Y,Y,Y,Y,Y


In [7]:
df = b.df(groups=["fast"], routines=["all"], funcs=["NRSCM" ,"RL-ZX", "Random-RL-ZX" , "cflow-opt", "gflow+cflow"], atts=["Qubits","Gates","Single Qubit", "2Q Count"], json_file="results/best_random_stats.json")

The function NRSCM has not been added. Call <benchmark>.show_attributes() to see loaded functions.
The function RL-ZX has not been added. Call <benchmark>.show_attributes() to see loaded functions.
The function Random-RL-ZX has not been added. Call <benchmark>.show_attributes() to see loaded functions.
The function gflow+cflow has not been added. Call <benchmark>.show_attributes() to see loaded functions.


In [9]:
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Enable LaTeX formatting with standard configuration
plt.rcParams['text.usetex'] = True

# Define the folders and full paths
base_path = '/data/'
folders = ["random_stats", "rl_stats"]
full_paths = {folder: os.path.join(base_path, folder) for folder in folders}

all_circuits = ["barenco_tof_3", "tof_3","Adder8", "barenco_tof_4", "csla_mux_3_original", "csum_mux_9_corrected","barenco_tof_5", "gf2^4_mult",
                "mod_mult_55", "mod5_4", "gf2^5_mult", "gf2^6_mult","qcla_adder_10", "qft_4","gf2^7_mult", "qcla_com_7","ham15-low", "vbe_adder_3","QFTAdd8", "grover_5"]
cflow = {
    "Adder8": 112, "barenco_tof_4": 37, "barenco_tof_5": 55, "gf2^4_mult": 94, "gf2^5_mult": 146, "gf2^6_mult": 209, "gf2^7_mult": 283, "gf2^8_mult": 383, "qcla_com_7": 131, "QFTAdd8": 175, "grover_5": 220,
    "vbe_adder_3": 36, "tof_3": 15, "qft_4": 44, "qcla_adder_10": 182, "mod_mult_55": 40, "csum_mux_9_corrected": 140, "mod5_4": 21, "ham15-low": 187, "csla_mux_3_original": 70, "barenco_tof_3": 20
}
circuits_name = {
    "Adder8": "adder$_8$", "barenco_tof_4": "barenco-tof$_4$", "barenco_tof_5": "barenco-tof$_5$", "gf2^4_mult": "gf($2^4$)-mult", "gf2^5_mult": "gf(${2^5}$)-mult",
    "gf2^6_mult": "gf(${2^6}$)-mult", "gf2^7_mult": "gf(${2^7}$)-mult", "gf2^8_mult": "gf(${2^8}$)-mult", "qcla_com_7": "qcla-com$_7$", "QFTAdd8": "QFTAdd$_8$", "grover_5": "Grover$_5$",
    "vbe_adder_3": "vbe-adder$_3$", "tof_3": "tof$_3$", "qft_4": "QFT$_4$", "qcla_adder_10": "qcla-adder$_{10}$", "mod_mult_55": "mod-mult$_{55}$", "mod5_4": "mod5$_4$", "ham15-low": "ham15-low",
    "csla_mux_3_original": "csla-mux$_3$", "barenco_tof_3": "barenco-tof$_3$", "csum_mux_9_corrected": "csum-mux$_9$"
}

# Function to read JSON files and extract data
def read_json_file(folder, circuit_name):
    if folder == "random_stats":
        specific_path = "_rand_policy_stats.json"
    else:
        specific_path = "_rl_stats.json"
    file_path = os.path.join(full_paths[folder], f"{circuit_name}{specific_path}")
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data["twoqubit"]
    return []

# Create a single figure with 10 rows and 4 columns
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(8, 8))
axes = axes.flatten()

# Initialize variables to store handles and labels for a shared legend
handles, labels = None, None

# Plot KDE or line for each circuit
for idx, circuit_name in enumerate(all_circuits):
    data_random = read_json_file("random_stats", circuit_name)
    data_rl = read_json_file("rl_stats", circuit_name)

    ax = axes[idx]  # Select subplot

    if data_random and len(set(data_random)) == 1:
        ax.axvline(data_random[0], label='random rl-zx', color='blue', linestyle='--')
    elif data_random:
        sns.kdeplot(
            data_random, label='random rl-zx',
            bw_adjust=1 if circuit_name in ['qft_4', 'csla_mux_3_original'] else 1,
            fill=True, ax=ax
        )

    if data_rl and len(set(data_rl)) == 1:
        ax.axvline(data_rl[0], label='rl-zx + cflow-zx', color='orange', linestyle='-.')
    elif data_rl:
        sns.kdeplot(
            data_rl, label='RL Policy',
            bw_adjust=8 if circuit_name in ['qft_4', 'csla_mux_3_original', 'vbe_adder_3'] else 1,
            fill=True, ax=ax
        )

    # Add a vertical red line for cflow
    if circuit_name in cflow:
        ax.axvline(cflow[circuit_name], label='cflow-zx', color='black', linestyle='--')

    # Store the handles and labels only once (from the first plot)
    if handles is None and labels is None:
        handles, labels = ax.get_legend_handles_labels()

    # Customize the subplot
    ax.set_title(rf'{circuits_name[circuit_name]}', fontsize=10)
    ax.set_xlabel(r'Twoqubits', fontsize=8)
    ax.set_ylabel(r'Density', fontsize=8)

# Add general titles for left and right groups
fig.text(0.27, 0.98, "a)", fontsize=16, fontweight='bold', ha='center')  # Left side
fig.text(0.77, 0.98, "b)", fontsize=16, fontweight='bold', ha='center')  # Right side

# Add a shared legend at the bottom center
fig.legend(handles, labels, loc='lower center', ncol=3, bbox_to_anchor=(0.5, 0.1), fontsize=10)


plt.tight_layout()
plt.subplots_adjust(top=0.96, bottom=0.19)


#plt.savefig("kde_wins_twoqubit.png", dpi=300)
plt.show()


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpqgq2bqyl 844f707e0917555faafa721cd5105dbd.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./844f707e0917555faafa721cd5105dbd.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

Error in callback <function _draw_all_if_interactive at 0x78dfd60cb400> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpbzp1n0ed 844f707e0917555faafa721cd5105dbd.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./844f707e0917555faafa721cd5105dbd.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpfssbyl2a 844f707e0917555faafa721cd5105dbd.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./844f707e0917555faafa721cd5105dbd.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

<Figure size 800x800 with 20 Axes>